# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.57'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\Spectra'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,10mol_FIE_R3,102.860045,1285.961207,941.651506,1285.961207,2649.210269,1.117623,20.526079,0.481638,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1mol_FIA_R1,102.759019,1286.323591,2523.054853,1286.323591,5393.650255,0.861581,15.551011,0.438305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1mol_FIA_R2,102.687955,1286.484421,2137.380302,1286.484421,4842.994280,0.982687,14.282850,0.235921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1mol_FIA_R3,102.733976,1286.544823,1935.499997,1286.544823,4672.379725,1.016852,19.287257,0.318345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1mol_FIB_R1,102.737776,1285.149925,2027.954498,1285.149925,5115.278798,1.057460,12.456320,0.333818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window', 'CRR'],
 sort=True)
diad_meta

['10mol_FID_R1.txt',
 '10mol_FID_R2.txt',
 '10mol_FID_R3.txt',
 '10mol_FIE_R1.txt',
 '10mol_FIE_R2.txt',
 '10mol_FIE_R3.txt',
 '1mol_FIA_R1.txt',
 '1mol_FIA_R2.txt',
 '1mol_FIA_R3.txt',
 '1mol_FIB_R1.txt',
 '1mol_FIB_R2.txt',
 '1mol_FIB_R3.txt',
 '4mol_FID_R1.txt',
 '4mol_FID_R2.txt',
 '4mol_FID_R3.txt',
 'AutoCalibration - Laser 532.05 nm - Grating 1800 - Coeff 0.00429475  PASS-postcrash.txt',
 'LL8_404a_r1.txt',
 'LL8_404a_r2.txt',
 'LL8_404a_r3.txt',
 'LL8_612_a_r1.txt',
 'LL8_612_a_r2.txt',
 'LL8_612_a_r3.txt',
 'LL8_612_c_r1.txt',
 'LL8_612_c_r2.txt',
 'LL8_612_c_r3.txt',
 'LL8_615_r1.txt',
 'LL8_615_r2.txt',
 'LL8_615_r3.txt',
 'LL8_622B_R1.txt',
 'LL8_622B_R2.txt',
 'LL8_622B_R3.txt',
 'LL8_624_r1.txt',
 'LL8_624_r2.txt',
 'LL8_624_r3.txt',
 'LL8_626_MI_r1.txt',
 'LL8_626_MI_r2.txt',
 'LL8_626_MI_r3.txt',
 'LL8_631_a_r1.txt',
 'LL8_631_a_r2.txt',
 'LL8_631_a_r3.txt',
 'LL8_631_b_r1.txt',
 'LL8_631_b_r2.txt',
 'LL8_631_b_r3.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.loop_convert_datastamp_to_metadata(path=spectra_path, 
files=diad_meta, creation=False,
modification=True)
meta

,filename,date,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,10mol_FIE_R1.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,1:46:39,6399,NaN
0,10mol_FIE_R2.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,1:49:45,6585,NaN
0,10mol_FIE_R3.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,1:53:30,6810,NaN
0,10mol_FID_R3.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:12:49,7969,NaN
0,1mol_FIB_R1.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:31:39,9099,NaN
0,1mol_FIB_R2.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:42:55,9775,NaN
0,1mol_FIB_R3.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:51:31,10291,NaN
0,10mol_FID_R1.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:5:15,7515,NaN
0,10mol_FID_R2.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,2:8:46,7726,NaN
0,1mol_FIA_R1.txt,"January 1, 2020",January,1,NaN,NaN,NaN,NaN,NaN,3:15:6,11706,NaN


## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

g:\my drive\berkeley_new\diadfit_outer\src\DiadFit\importing_data_files.py:1065: FutureWarning: The default value of regex will change from True to False in a future version.
  names=names.str.replace(file_type, '')


'10mol_FIE_R1'

## Now get filename from spectra into same form as metadata

In [9]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

In [10]:
file_s

['10mol_FIE_R3',
 '1mol_FIA_R1',
 '1mol_FIA_R2',
 '1mol_FIA_R3',
 '1mol_FIB_R1',
 '1mol_FIB_R2',
 '1mol_FIB_R3',
 '4mol_FID_R1',
 '4mol_FID_R2',
 '4mol_FID_R3',
 'LL8_404a_r1',
 'LL8_404a_r2',
 'LL8_404a_r3',
 'LL8_612_a_r1',
 'LL8_612_a_r2',
 'LL8_612_a_r3',
 'LL8_612_c_r1',
 'LL8_612_c_r2',
 'LL8_612_c_r3',
 'LL8_615_r1',
 'LL8_615_r2',
 'LL8_615_r3',
 'LL8_622B_R1',
 'LL8_622B_R2',
 'LL8_622B_R3',
 'LL8_624_r1',
 'LL8_624_r2',
 'LL8_624_r3',
 'LL8_626_MI_r1',
 'LL8_626_MI_r2',
 'LL8_626_MI_r3',
 'LL8_631_a_r1',
 'LL8_631_a_r2',
 'LL8_631_a_r3',
 'LL8_631_b_r1',
 'LL8_631_b_r2',
 'LL8_631_b_r3']

In [11]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [12]:
df_combo.head()

,Unnamed: 0,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,0,10mol_FIE_R3,102.860045,1285.961207,941.651506,1285.961207,2649.210269,1.117623,20.526079,0.481638,...,January,1,NaN,NaN,NaN,NaN,NaN,1:53:30,6810,NaN
1,0,1mol_FIA_R1,102.759019,1286.323591,2523.054853,1286.323591,5393.650255,0.861581,15.551011,0.438305,...,January,1,NaN,NaN,NaN,NaN,NaN,3:15:6,11706,NaN
2,0,1mol_FIA_R2,102.687955,1286.484421,2137.380302,1286.484421,4842.994280,0.982687,14.282850,0.235921,...,January,1,NaN,NaN,NaN,NaN,NaN,3:22:7,12127,NaN
3,0,1mol_FIA_R3,102.733976,1286.544823,1935.499997,1286.544823,4672.379725,1.016852,19.287257,0.318345,...,January,1,NaN,NaN,NaN,NaN,NaN,3:28:32,12512,NaN
4,0,1mol_FIB_R1,102.737776,1285.149925,2027.954498,1285.149925,5115.278798,1.057460,12.456320,0.333818,...,January,1,NaN,NaN,NaN,NaN,NaN,2:31:39,9099,NaN


In [13]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [14]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

ValueError: A value in x_new is below the interpolation range.

In [15]:
df_combo.to_excel('FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [24]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None


In [27]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')



In [29]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo

### Add some more useful columns

In [30]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [31]:
df_combo_sec_phase.to_excel('FI_fitting_w_sec_phases.xlsx')